In [21]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import random
from random import randint
from sklearn.model_selection import StratifiedKFold

save_results_to = '/home/hnoghabi/EGFR/'
torch.manual_seed(42)
random.seed(42)

GDSCE = pd.read_csv("GDSC_exprs.z.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)

GDSCM = pd.read_csv("GDSC_mutations.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:,~GDSCM.columns.duplicated()]

GDSCC = pd.read_csv("GDSC_CNA.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:,~GDSCC.columns.duplicated()]

PDXEerlo = pd.read_csv("PDX_exprs.Erlotinib.eb_with.GDSC_exprs.Erlotinib.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXEerlo = pd.DataFrame.transpose(PDXEerlo)
# PDXMerlo = pd.read_csv("PDX_mutations.Erlotinib.tsv", 
#                    sep = "\t", index_col=0, decimal = ".")
PDXMerlo = pd.read_csv("PDX_mutations.Erlotinib - Copy.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXMerlo = pd.DataFrame.transpose(PDXMerlo)
# PDXCerlo = pd.read_csv("PDX_CNA.Erlotinib.tsv", 
#                    sep = "\t", index_col=0, decimal = ".")
PDXCerlo = pd.read_csv("PDX_CNV.Erlotinib.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXCerlo.drop_duplicates(keep='last')
PDXCerlo = pd.DataFrame.transpose(PDXCerlo)
PDXCerlo = PDXCerlo.loc[:,~PDXCerlo.columns.duplicated()]

PDXEcet = pd.read_csv("PDX_exprs.Cetuximab.eb_with.GDSC_exprs.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXEcet = pd.DataFrame.transpose(PDXEcet)
# PDXMcet = pd.read_csv("PDX_mutations.Cetuximab.tsv", 
#                    sep = "\t", index_col=0, decimal = ".")
PDXMcet = pd.read_csv("PDX_mutations.Cetuximab - Copy.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXMcet = pd.DataFrame.transpose(PDXMcet)
# PDXCcet = pd.read_csv("PDX_CNA.Cetuximab.tsv", 
#                    sep = "\t", index_col=0, decimal = ".")
PDXCcet = pd.read_csv("PDX_CNV.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXCcet.drop_duplicates(keep='last')
PDXCcet = pd.DataFrame.transpose(PDXCcet)
PDXCcet = PDXCcet.loc[:,~PDXCcet.columns.duplicated()]

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1
GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(GDSCC.columns)
ls = ls.intersection(PDXEerlo.columns)
ls = ls.intersection(PDXMerlo.columns)
ls = ls.intersection(PDXCerlo.columns)
ls = ls.intersection(PDXEcet.columns)
ls = ls.intersection(PDXMcet.columns)
ls = ls.intersection(PDXCcet.columns)
ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCC.index)
ls3 = PDXEerlo.index.intersection(PDXMerlo.index)
ls3 = ls3.intersection(PDXCerlo.index)
ls4 = PDXEcet.index.intersection(PDXMcet.index)
ls4 = ls4.intersection(PDXCcet.index)
ls = pd.unique(ls)

PDXEerlo = PDXEerlo.loc[ls3,ls]
PDXMerlo = PDXMerlo.loc[ls3,ls]
PDXCerlo = PDXCerlo.loc[ls3,ls]
PDXEcet = PDXEcet.loc[ls4,ls]
PDXMcet = PDXMcet.loc[ls4,ls]
PDXCcet = PDXCcet.loc[ls4,ls]
GDSCE = GDSCE.loc[:,ls]
GDSCM = GDSCM.loc[:,ls]
GDSCC = GDSCC.loc[:,ls]

GDSCR = pd.read_csv("GDSC_response.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")

GDSCR.rename(mapper = str, axis = 'index', inplace = True)

d = {"R":0,"S":1}
GDSCR["response"] = GDSCR.loc[:,"response"].apply(lambda x: d[x])

responses = GDSCR
drugs = set(responses["drug"].values)
exprs_z = GDSCE
cna = GDSCC
mut = GDSCM
expression_zscores = []
CNA=[]
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"]==drug,:].index.values
    e_z = exprs_z.loc[samples,:]
    c = cna.loc[samples,:]
    m = mut.loc[samples,:]
    m = mut.loc[samples,:]
    # next 3 rows if you want non-unique sample names
    e_z.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    c.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    m.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    expression_zscores.append(e_z)
    CNA.append(c)
    mutations.append(m)
responses.index = responses.index.values +"_"+responses["drug"].values
GDSCEv2 = pd.concat(expression_zscores, axis =0 )
GDSCCv2 = pd.concat(CNA, axis =0 )
GDSCMv2 = pd.concat(mutations, axis =0 )
GDSCRv2 = responses

ls2 = GDSCEv2.index.intersection(GDSCMv2.index)
ls2 = ls2.intersection(GDSCCv2.index)
GDSCEv2 = GDSCEv2.loc[ls2,:]
GDSCMv2 = GDSCMv2.loc[ls2,:]
GDSCCv2 = GDSCCv2.loc[ls2,:]
GDSCRv2 = GDSCRv2.loc[ls2,:]

Y = GDSCRv2['response'].values

PDXRcet = pd.read_csv("PDX_response.Cetuximab.tsv", 
                       sep = "\t", index_col=0, decimal = ",")
PDXRcet.loc[PDXRcet.iloc[:,1] == 'R'] = 0
PDXRcet.loc[PDXRcet.iloc[:,1] == 'S'] = 1
PDXRcet = PDXRcet.loc[ls4,:]
Ytscet = PDXRcet['response'].values    

PDXRerlo = pd.read_csv("PDX_response.Erlotinib.tsv", 
                       sep = "\t", index_col=0, decimal = ",")
PDXRerlo.loc[PDXRerlo.iloc[:,1] == 'R'] = 0
PDXRerlo.loc[PDXRerlo.iloc[:,1] == 'S'] = 1
PDXRerlo = PDXRerlo.loc[ls3,:]
Ytserlo = PDXRerlo['response'].values  

hdm1 = 32
hdm2 = 16
hdm3 = 256
rate1 = 0.5
rate2 = 0.8
rate3 = 0.5
rate4 = 0.3



In [22]:
scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEerlo = scalerGDSC.transform(PDXEerlo.values)    
X_testEcet = scalerGDSC.transform(PDXEcet.values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_trainC = np.nan_to_num(GDSCCv2.values)
X_testMerlo = np.nan_to_num(PDXMerlo.values)
X_testCerlo = np.nan_to_num(PDXCerlo.values)
X_testMcet = np.nan_to_num(PDXMcet.values)
X_testCcet = np.nan_to_num(PDXCcet.values)

TX_testEerlo = torch.FloatTensor(X_testEerlo)
TX_testMerlo = torch.FloatTensor(X_testMerlo)
TX_testCerlo = torch.FloatTensor(X_testCerlo)
ty_testEerlo = torch.FloatTensor(Ytserlo.astype(int))

TX_testEcet = torch.FloatTensor(X_testEcet)
TX_testMcet = torch.FloatTensor(X_testMcet)
TX_testCcet = torch.FloatTensor(X_testCcet)
ty_testEcet = torch.FloatTensor(Ytscet.astype(int))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
h_dim3 = hdm3        
Z_in = h_dim1 + h_dim2 + h_dim3

costtr = []
auctr = []
costts = []
aucts = []

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output

class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output    


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.ReLU(),
            nn.Dropout(rate3))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate4),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load('EGFRv2Exprs.pt')
AutoencoderM = torch.load('EGFRv2Mut.pt')
AutoencoderC = torch.load('EGFRv2CNA.pt')

Clas = torch.load('EGFRv2Class.pt')

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCt = roc_auc_score(Y, PredX.detach().numpy())
print(AUCt)

ZETerlo = AutoencoderE(TX_testEerlo)
ZMTerlo = AutoencoderM(TX_testMerlo)
ZCTerlo = AutoencoderC(TX_testCerlo)
ZTTerlo = torch.cat((ZETerlo, ZMTerlo, ZCTerlo), 1)
ZTTerlo = F.normalize(ZTTerlo, p=2, dim=0)
PredTerlo = Clas(ZTTerlo)
AUCterlo = roc_auc_score(Ytserlo, PredTerlo.detach().numpy())
print(AUCterlo)

ZETcet = AutoencoderE(TX_testEcet)
ZMTcet = AutoencoderM(TX_testMcet)
ZCTcet = AutoencoderC(TX_testCcet)
ZTTcet = torch.cat((ZETcet, ZMTcet, ZCTcet), 1)
ZTTcet = F.normalize(ZTTcet, p=2, dim=0)
PredTcet = Clas(ZTTcet)
AUCtcet = roc_auc_score(Ytscet, PredTcet.detach().numpy())
print(AUCtcet)

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0.9440556750399118
0.7222222222222222
0.8
